## Load Packages

In [1]:
import os

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import json
import requests

import time

import re

from lxml import html

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Load API Scraped Discussion Data

In [2]:
post_df = pd.read_excel('ASHRAE-Kaggle_WebScraping.xlsx', sheet_name = 'discussion', index_col = 0)

In [3]:
print(post_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 0 to 466
Data columns (total 24 columns):
totalRows              374 non-null int64
authorType             374 non-null object
commentCount           374 non-null int64
currentUserVote        374 non-null int64
downvoteUrl            374 non-null object
id                     374 non-null int64
isKernel               374 non-null bool
isPrivate              374 non-null bool
isSticky               374 non-null bool
kernelUrl              0 non-null float64
lastCommenterType      374 non-null object
lastCommenterName      371 non-null object
lastCommenterUrl       374 non-null object
lastCommentPostDate    374 non-null datetime64[ns]
medal                  265 non-null object
parentName             374 non-null object
parentUrl              374 non-null object
postDate               374 non-null datetime64[ns]
title                  374 non-null object
topicUrl               374 non-null object
upvoteUrl              374 non

## Clean dataframe

In [4]:
# Look for and drop duplicate rows
post_df.drop_duplicates(subset=['id'], inplace=True)

# Change format of post time to date/time format
post_df['postDate'] = pd.to_datetime(post_df['postDate'])

print(post_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 0 to 465
Data columns (total 24 columns):
totalRows              293 non-null int64
authorType             293 non-null object
commentCount           293 non-null int64
currentUserVote        293 non-null int64
downvoteUrl            293 non-null object
id                     293 non-null int64
isKernel               293 non-null bool
isPrivate              293 non-null bool
isSticky               293 non-null bool
kernelUrl              0 non-null float64
lastCommenterType      293 non-null object
lastCommenterName      290 non-null object
lastCommenterUrl       293 non-null object
lastCommentPostDate    293 non-null datetime64[ns]
medal                  220 non-null object
parentName             293 non-null object
parentUrl              293 non-null object
postDate               293 non-null datetime64[ns]
title                  293 non-null object
topicUrl               293 non-null object
upvoteUrl              293 non

In [5]:
# Add author column - extract from userAvatar field
post_df['author'] = post_df['userAvatar'].str.split(':').str[1]
post_df['author'] = post_df['author'].str[2:]
post_df['author'] = post_df['author'].str.split('\'').str[0]

print(post_df['author'])

0      Konstantin Yakovlev
1                  Clayton
2            Chris Balbach
6            Alexey Pronin
7            Heng CherKeng
10           Chris Balbach
13           Vitalii Mokin
16         Robert Stockton
18            Bojan Tunguz
19            Gunes Evitan
60         Victor_Palacios
62                  chmaxx
63        Geoffrey Bolmier
65     Konstantin Yakovlev
66                     S D
70               Poe Dator
73         Dmitry Labazkin
75            Ashish Gupta
76                Serigne 
79          Hiroyuki Namba
80                  ryches
82                Joe Eddy
83          Dhananjay Raut
84                senkin13
89                     S D
90            Gunes Evitan
92             Sohier Dane
94            Gunes Evitan
100           Gunes Evitan
101                Nanashi
              ...         
425                 Jie Wu
426        Udbhav Pangotra
427              Waylon Wu
428               XLCBobby
429                  Rahul
430            Nathan Zhai
4

## Scrape discussion and comments text

In [6]:
def discussionPlusComments(post_id):
    # URL to scrape from taking in user input
    url = "https://www.kaggle.com/c/ashrae-energy-prediction/discussion/" + str(post_id)
    dcap = dict(DesiredCapabilities.PHANTOMJS)
    dcap["phantomjs.page.settings.userAgent"] = ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36 OPR/44.0.2510.857")
    # Use headless chrome as the webdriver
    driver = webdriver.Chrome(desired_capabilities=dcap, service_args=['--ignore-ssl-errors=true','--ssl-protocol=any'])
    driver.get(url)
    print(url)
    # Wait for page to load
    time.sleep(5)
    # Save a screenshot of the page being scraped
    #driver.save_screenshot(r'image.png')
    s = BeautifulSoup(driver.page_source, "lxml")
    pretty_s = s.prettify()
    #print(pretty_s)
    driver.close()
    
    # Find the html of interest with the following tag
    post_tags = s.findAll('div', 'topic__content')
    post = []
    for tag in post_tags:
        post.append(tag.text)
    
    # Find the html of interest with the tag corresponding to comment text
    comment_tags = s.findAll('div', 'discussion-comment__body')
    comments_content = []
    for tag in comment_tags:
        comments_content.append(tag.text)
    #print(len(comments_content))
    
    # Find the html of interest with the tag corresponding to comment author
    comment_tags = s.findAll('div', 'discussion-comment__author')
    comments_author = []
    for tag in comment_tags:
        for img in tag.find_all('img', alt=True):
            if 'tier' not in img['alt']: 
                comments_author.append(img['alt'])
    #print(comments_author)
    #print(len(comments_author))
    
    # Find the html of interest with the tag corresponding to comment post time
    comment_tags = s.findAll('span', title=re.compile("GMT"))
    comments_time = []
    for tag in comment_tags:
        try:
            comments_time.append(tag['title'])
        except:
            continue
    #print(len(comments_time[2:]))
    
    # Create a dataframe for the comments
    comments_df = pd.DataFrame({'content':comments_content,'id':post_id,'postDate':comments_time[2:],'author':comments_author})
    
    return post[0], comments_df

In [8]:
# Create a dataframe to store comments data
comments_df = pd.DataFrame(columns = ['content','id','postDate','author'])

# For every discussion 'id', scrape page and extract post content
for post_id in post_df['id']:
    post,comments = discussionPlusComments(post_id)
    post_df.at[post_df.index[post_df['id'] == post_id], 'content'] = post
    comments_df = pd.concat([comments_df, comments], ignore_index=True)

# Modify string format of comments_df 'post_time' column to be date/time suitable
comments_df['postDate'] = comments_df['postDate'].str[4:]
comments_df['postDate'] = comments_df['postDate'].str.split('GMT').str[0] # As IST is the same as GMT (Winter)
comments_df['postDate'] = pd.to_datetime(comments_df['postDate'], format = '%b %d %Y %H:%M:%S ')

https://www.kaggle.com/c/ashrae-energy-prediction/discussion/112872


KeyboardInterrupt: 

In [ ]:
# Write the scraped data to the csv database
post_df.to_csv('Section2/discussions.csv', sep=',', header=True, index=False)
comments_df.to_csv('Section2/comments.csv', sep=',', header=True, index=False)